In [1]:
import os
import torch
import time
import numpy as np
import trimesh
from progressbar import *
from sklearn import preprocessing
from utils.util import load_obj, write_pickle_file, get_obj_vertex_ali
from utils.generate_normal import readFaceInfo, generate_normal

In [2]:
data_root = 'data/Future-3D-Reconstruction'
model_dir = os.path.join(data_root, 'train', 'model')
verts_normals_dir = os.path.join(data_root, 'train', 'verts_normals')

In [3]:
model_IDs = os.listdir(model_dir)
model_IDs = [ID for ID in model_IDs if ID[-4:] == '.obj']
print(len(model_IDs))

5203


In [4]:
def sample_points_from_mesh(obj_path, num_verts=None):
    obj_dict = load_obj(obj_path)
    mesh = trimesh.Trimesh(obj_dict['vertices'], obj_dict['faces'], process=False)
    
    if num_verts is None:
        num_verts = mesh.vertices.shape[0]
    points, index = trimesh.sample.sample_surface(mesh, num_verts)

    triangles = mesh.triangles[index]
    pt1 = triangles[:,0,:]
    pt2 = triangles[:,1,:]
    pt3 = triangles[:,2,:]
    normal = np.cross(pt3-pt1, pt2-pt1)
    normal = preprocessing.normalize(normal, axis=1)

    return points, normal

In [5]:
def get_verts_and_normals(obj_path):
    obj_dict = load_obj(obj_path)
    verts = obj_dict['vertices']
    vns = obj_dict['vns']
    faces = obj_dict['faces']
    faces_vns = obj_dict['faces_vns']
    
    normals = np.zeros_like(verts)
    for i in range(faces.shape[0]):
        normals[faces[i]] += vns[faces_vns[i]]
    normals = preprocessing.normalize(normals, axis=1)
    return verts, normals

In [7]:
for fn in os.listdir(verts_normals_dir):
    os.remove(os.path.join(verts_normals_dir, fn))

In [ ]:
widgets = ['Progress: ', Percentage(), ' ', Bar('#'), ' ', Timer(), ' ', ETA()]
total = len(model_IDs)
pbar = ProgressBar(widgets=widgets, maxval=10*total).start()
for i, model_ID in enumerate(model_IDs):
    model_path = os.path.join(model_dir, model_ID)
    verts_normals_path = os.path.join(verts_normals_dir, model_ID.replace('.obj', '.pkl'))
    #verts, normals = sample_points_from_mesh(model_path, num_verts=16384)
    try:
        verts, normals = get_verts_and_normals(model_path)
        write_pickle_file(verts_normals_path, {'verts': verts, 'normals': normals})
    except:
        continue
    pbar.update(10 * i + 1)
pbar.finish()

In [ ]:
widgets = ['Progress: ', Percentage(), ' ', Bar('#'), ' ', Timer(), ' ', ETA()]
total = len(model_IDs)
pbar = ProgressBar(widgets=widgets, maxval=10*total).start()
for i, model_ID in enumerate(model_IDs):
    model_path = os.path.join(model_dir, model_ID)
    verts_normals_path = os.path.join(verts_normals_dir, model_ID.replace('.obj', '.pth'))
    face_pts, face_axis = readFaceInfo(model_path)
    #print(face_pts.shape, face_axis.shape)
    verts = get_obj_vertex_ali(model_path)
    #print(verts.shape)
    verts, normals = generate_normal(verts, face_pts, face_axis)
    #print(verts.shape, normals.shape)
    write_pickle_file(verts_normals_path, {'verts': verts, 'normals': normals})
    pbar.update(10 * i + 1)
pbar.finish()